In [ ]:
import numpy as np
import pandas as pd
from numba import njit
from statsmodels.tsa.stattools import adfuller
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

@njit
def calculate_weights_numba(d, thresh):
    w = [1.0]
    k = 1
    while True:
        w_ = -w[-1] * (d - k + 1) / k
        if abs(w_) < thresh:
            break
        w.append(w_)
        k += 1
    return np.array(w[::-1])

@njit
def frac_diff_numba(series_values, w, width):
    n = len(series_values)
    diff_series = np.empty(n)
    diff_series[:] = np.nan
    for i in range(width, n):
        diff_value = 0.0
        for j in range(width):
            diff_value += w[j] * series_values[i - width + j]
        diff_series[i] = diff_value
    return diff_series

def frac_diff_optimized(series, d, thresh=1e-5):
    """
    Aplica diferenciação fracionária à série temporal utilizando Numba para otimização.

    Parâmetros:
    - series: pd.Series, a série temporal a ser diferenciada.
    - d: float, ordem de diferenciação fracionária (0 < d < 1).
    - thresh: float, limiar para os coeficientes binomiais.

    Retorna:
    - pd.Series, série diferenciada fracionariamente.
    """
    # Calcular os coeficientes binomiais
    w = calculate_weights_numba(d, thresh)
    width = len(w)
    
    # Converter a série para um array numpy
    series_values = series.values
    n = len(series_values)
    
    # Aplicar a diferenciação fracionária utilizando Numba
    diff_series = frac_diff_numba(series_values, w, width)
    
    return pd.Series(diff_series, index=series.index)

def adf_test(series, title='ADF Test'):
    """
    Realiza o teste de Dickey-Fuller aumentado e imprime os resultados.

    Parâmetros:
    - series: pd.Series, a série temporal a ser testada.
    - title: str, título para identificar o teste.

    Retorna:
    - None
    """
    print(f'\n==== {title} ====')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['Estatística ADF', 'p-valor', 'Número de Lags Usados', 'Número de Observações Usadas']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Valor Crítico ({key})'] = value
    print(out.to_string())
    if result[1] <= 0.05:
        print("Evidência forte contra a hipótese nula (presença de raiz unitária), rejeitamos a hipótese nula.")
    else:
        print("Evidência fraca contra a hipótese nula, não rejeitamos a hipótese nula.")

def create_lag_matrix(series, n_lags):
    """
    Cria uma matriz de lags para a série temporal.

    Parâmetros:
    - series: pd.Series, a série temporal.
    - n_lags: int, número de lags a serem criados.

    Retorna:
    - X: np.ndarray, matriz de lags.
    - y: np.ndarray, variável alvo ajustada.
    """
    # Criar um DataFrame com as lags
    df_lags = pd.concat([series.shift(i) for i in range(1, n_lags + 1)], axis=1)
    df_lags.columns = [f'lag_{i}' for i in range(1, n_lags + 1)]
    
    # Remover linhas com NaN causados pelo shift
    df_lags = df_lags.dropna()
    
    # Variável alvo
    y = series.loc[df_lags.index].values
    
    # Converter para array NumPy
    X = df_lags.values
    
    return X, y

def monteCarlo(serie_predita, constant, params, order):
    out = len(serie_predita)
    Y = np.zeros(out)
    Y[:(order)] = serie_predita.iloc[:(order)]

    for i in range(order, out):
        # Extract the relevant data and transpose if needed
        data = serie_predita.iloc[i-order:i].values
        phi_transpose = np.transpose(params)

        Y[i] = data @ phi_transpose + constant

    Y = pd.DataFrame(Y)
    Y.index = serie_predita.index
    Y.rename(columns={0: 'fraqdiff_pred'}, inplace=True)
    return Y

import statsmodels.api as sm
import numpy as np

def auto_reg(order, serie_predita):
    """
    Ajusta um modelo autoregressivo e retorna previsões alinhadas com a série original.

    Parameters:
    - order (int): Ordem do modelo autoregressivo.
    - serie_predita (pd.Series): Série temporal para ajustar o modelo.

    Returns:
    - Y_pred_aligned (pd.Series): Série de previsões alinhadas com a série original.
    - constant (float): Constante do modelo.
    - params (pd.Series): Parâmetros do modelo.
    """
    # Garantir que serie_predita é uma pandas Series
    if not isinstance(serie_predita, pd.Series):
        serie_predita = pd.Series(serie_predita)
    
    # Criar matriz de lags sem usar np.roll para evitar deslocamento circular
    X = pd.concat([serie_predita.shift(i+1) for i in range(order)], axis=1)
    X.columns = [f'lag_{i+1}' for i in range(order)]
    
    # Remover linhas com NaN devido aos lags
    X = X.dropna()
    y = serie_predita.loc[X.index]
    
    # Adicionar constante
    X = sm.add_constant(X)
    
    # Ajustar o modelo autoregressivo
    model = sm.OLS(y, X)
    result = model.fit()
    
    # Exibir o resumo do modelo
    print(result.summary())
    
    # Obter parâmetros
    constant = result.params['const']
    params = result.params.drop('const')
    
    # Prever os valores in-sample
    Y_pred = result.predict(X)
    
    # Garantir que Y_pred está alinhado com y
    Y_pred_aligned = Y_pred.copy()
    Y_pred_aligned.index = y.index
    
    return Y_pred_aligned, constant, params


In [ ]:
series = pd.read_parquet("../output/dollar-bars-[10000000]/dollar_bars.parquet/part.0.parquet")
series.head()

In [ ]:
import pandas as pd
import numpy as np
from numba import njit
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Função de diferenciação fracionária otimizada com Numba
@njit
def frac_diff_numba(series, w, width):
    diff_series = np.zeros(len(series))
    for i in range(width, len(series)):
        value = 0.0
        for j in range(width):
            value += w[j] * series[i - j]
        diff_series[i] = value
    return diff_series

def frac_diff_optimized(series, d, max_lag=100):
    # Calcula os pesos da diferenciação fracionária
    w = [1.0]
    for k in range(1, max_lag):
        w_ = -w[-1] * (d - k + 1) / k
        w.append(w_)
        if abs(w[-1]) < 1e-5:
            break
    width = len(w)
    series_values = series.values.astype(np.float64)  # Assegura que os valores são float64
    diff_series = frac_diff_numba(series_values, np.array(w, dtype=np.float64), width)
    return pd.Series(diff_series, index=series.index)

# Função para realizar o teste ADF
def adf_test(series, title=''):
    result = adfuller(series, autolag='AIC')
    print(f'== {title} ==')
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    for key, value in result[4].items():
        print(f'Critical Value {key}: {value}')
    print('')

# Função para criar matriz de lags
def create_lag_matrix(series, n_lags):
    X = pd.concat([series.shift(i) for i in range(1, n_lags + 1)], axis=1)
    y = series
    X = X.dropna()
    y = y.loc[X.index]
    return X.values, y.values

# Função autoregressiva (exemplo simples)
def auto_reg(order, series):
    from statsmodels.tsa.ar_model import AutoReg
    model = AutoReg(series, lags=order, old_names=False)
    model_fit = model.fit()
    Y_pred = model_fit.predict(start=order, end=len(series)-1)
    constant = model_fit.params[0]
    params = model_fit.params[1:]
    return Y_pred, constant, params

# Leitura da série
series = pd.read_parquet("../output/dollar-bars-[10000000]/dollar_bars.parquet/part.0.parquet")
print(series.head())

# Definir o valor de diferenciação fracionária
d = 0.3

# Aplicar a diferenciação fracionária na coluna 'price_close' em vez do DataFrame completo
print("Aplicando diferenciação fracionária na série 'price_close'...")
series_frac_diff = frac_diff_optimized(series['price_close'], d).dropna()
print("Diferenciação fracionária concluída.")

# Realizar o teste ADF na série diferenciada completa
adf_test(series_frac_diff, title=f'Preço de Fechamento (price_close) - Diferenciação Fracionária d={d}')

# Exibir as primeiras linhas da série diferenciada
display(series_frac_diff.head())

# Definir o tamanho da amostra (5% dos dados)
sample_size = int(len(series_frac_diff) * 0.05)  # Aumentado para 5%
series_frac_diff_sample = series_frac_diff.sample(n=sample_size, random_state=42).sort_index()

# Realizar o teste ADF na amostra de 5%
adf_test(series_frac_diff_sample, title=f'Preço de Fechamento (price_close) - Diferenciação Fracionária d={d} (5% Amostra)')

# Exibir as primeiras linhas da amostra diferenciada
display(series_frac_diff_sample.head())

# Criar matriz de lags
n_lags = 5
X, y = create_lag_matrix(series_frac_diff, n_lags)
print(f"Matriz de lags criada com forma: {X.shape}")
print(f"Variável alvo (y) criada com forma: {y.shape}")

# Aplicar PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA()
X_pca = pca.fit_transform(X_scaled)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
print("Variância explicada acumulada:", cumulative_variance)

# Determinar número de componentes
n_components_var = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Número ótimo de componentes (95% da variância explicada): {n_components_var}")

# Reduzir dimensionalidade
n_components = n_components_var
pca_reduced = PCA(n_components=n_components)
X_reduced = pca_reduced.fit_transform(X_scaled)
print(f"Forma da matriz após PCA: {X_reduced.shape}")

# Aplicação em modelagem
model = LinearRegression()
scores = cross_val_score(model, X_reduced, y, cv=5, scoring='neg_mean_squared_error')
print(f"MSE médio (5-fold CV): {-np.mean(scores):.4f}")

# Plotar ACF e PACF
desired_lags = 40
sample_size = len(series_frac_diff_sample)
max_allowed_lags = sample_size // 2 - 1
adjusted_lags = min(desired_lags, max_allowed_lags)

print(f"Usando lags={adjusted_lags} para a amostra de tamanho {sample_size}")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Série Completa
plot_acf(series_frac_diff, ax=axes[0, 0], lags=desired_lags, zero=False)
axes[0, 0].set_title(f'ACF - Série Completa (d={d})')

plot_pacf(series_frac_diff, ax=axes[1, 0], lags=desired_lags, zero=False, method='ywm')
axes[1, 0].set_title(f'PACF - Série Completa (d={d})')

# Amostra de 5%
plot_acf(series_frac_diff_sample, ax=axes[0, 1], lags=adjusted_lags, zero=False)
axes[0, 1].set_title(f'ACF - 5% Amostra (d={d})')

plot_pacf(series_frac_diff_sample, ax=axes[1, 1], lags=adjusted_lags, zero=False, method='ywm')
axes[1, 1].set_title(f'PACF - 5% Amostra (d={d})')

plt.tight_layout()
plt.show()

# Aplicar a função autoregressiva
order = 1  # Defina a ordem do modelo autoregressivo
Y_pred, constant, params = auto_reg(order, series_frac_diff)

# Visualizar a série prevista
plt.figure(figsize=(14,6))
plt.plot(series_frac_diff, label='Série Diferenciada Fracionária', alpha=0.5)
plt.plot(Y_pred, label='Série Prevista', alpha=0.7)
plt.legend()
plt.title('Comparação entre Série Diferenciada e Série Prevista')
plt.xlabel('Data')
plt.ylabel('Preço Diferenciado')
plt.show()


# Resíduos

In [ ]:
import pandas as pd

# Converter y para pandas Series
y_series = pd.Series(y, name='y_true')

# Verificar se Y_pred é um DataFrame com uma única coluna
if isinstance(Y_pred, pd.DataFrame):
    if Y_pred.shape[1] == 1:
        # Converter para Series
        Y_pred_series = Y_pred.iloc[:, 0].reset_index(drop=True)
        Y_pred_series.name = 'y_pred'
        print(f"Convertido Y_pred para Series com comprimento {len(Y_pred_series)}.")
    else:
        raise ValueError("Y_pred possui mais de uma coluna. Por favor, selecione a coluna correta para as previsões.")
elif isinstance(Y_pred, pd.Series):
    Y_pred_series = Y_pred.reset_index(drop=True)
    Y_pred_series.name = 'y_pred'
    print(f"Y_pred já é uma Series com comprimento {len(Y_pred_series)}.")
else:
    raise TypeError("Y_pred deve ser uma pandas Series ou DataFrame.")

# Verificar comprimentos
len_y = len(y_series)
len_Y_pred = len(Y_pred_series)

print(f"Comprimento de y_series: {len_y}")
print(f"Comprimento de Y_pred_series: {len_Y_pred}")

# Calcular a diferença de comprimento
diff = len_Y_pred - len_y

if diff > 0:
    print(f"Y_pred_series é {diff} pontos maior que y_series. Truncando os últimos {diff} pontos de Y_pred_series.")
    Y_pred_aligned = Y_pred_series.iloc[:-diff].reset_index(drop=True)
    y_aligned = y_series.reset_index(drop=True)
elif diff < 0:
    print(f"Y_pred_series é {abs(diff)} pontos menor que y_series. Truncando os últimos {abs(diff)} pontos de y_series.")
    y_aligned = y_series.iloc[:len_Y_pred].reset_index(drop=True)
    Y_pred_aligned = Y_pred_series.reset_index(drop=True)
else:
    print("y_series e Y_pred_series já estão alinhadas em comprimento.")
    y_aligned = y_series.reset_index(drop=True)
    Y_pred_aligned = Y_pred_series.reset_index(drop=True)

print(f"Comprimento de y_aligned: {len(y_aligned)}")
print(f"Comprimento de Y_pred_aligned: {len(Y_pred_aligned)}")


# Calcular os resíduos
residuals = y_aligned - Y_pred_aligned

# Exibir as primeiras linhas dos resíduos
display(residuals.head())

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plotar os resíduos ao longo do tempo
plt.figure(figsize=(14,6))
plt.plot(residuals, label='Resíduos (y_true - Y_pred)')
plt.axhline(0, color='red', linestyle='--')
plt.legend()
plt.title('Resíduos da Previsão Autoregressiva')
plt.xlabel('Observações')
plt.ylabel('Resíduo')
plt.show()

# Estatísticas descritivas dos resíduos
print("Estatísticas dos Resíduos:")
print(residuals.describe())

# Plotar histograma dos resíduos
plt.figure(figsize=(10,5))
sns.histplot(residuals, kde=True, bins=50)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduo')
plt.ylabel('Frequência')
plt.show()

# Teste de normalidade (Shapiro-Wilk)
# Shapiro-Wilk tem limite de amostra (normalmente até 5000 observações)
sample_size = min(5000, len(residuals))
sample_residuals = residuals.sample(n=sample_size, random_state=42)
stat, p_value = shapiro(sample_residuals)
print(f"Estatística Shapiro-Wilk: {stat:.4f}, p-valor: {p_value:.4f}")
if p_value > 0.05:
    print("Os resíduos parecem seguir uma distribuição normal (não rejeita H0).")
else:
    print("Os resíduos não seguem uma distribuição normal (rejeita H0).")

# Autocorrelação dos resíduos
fig, ax = plt.subplots(1, 2, figsize=(15,5))
plot_acf(residuals, ax=ax[0], lags=40, zero=False)
ax[0].set_title('ACF dos Resíduos')
plot_pacf(residuals, ax=ax[1], lags=40, zero=False, method='ywm')
ax[1].set_title('PACF dos Resíduos')
plt.show()



# Cusum Simétrico

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Supondo que 'residuals' é um pandas Series obtido anteriormente
# Certifique-se de que os resíduos estão ordenados corretamente

# Inicializar as colunas de CUSUM positivo e negativo
cusum_positive = [0]
cusum_negative = [0]

# Iterar sobre os resíduos para calcular o CUSUM
for residual in residuals:
    # CUSUM positivo: soma acumulativa das diferenças positivas
    s_pos = max(0, cusum_positive[-1] + residual)
    cusum_positive.append(s_pos)
    
    # CUSUM negativo: soma acumulativa das diferenças negativas
    s_neg = min(0, cusum_negative[-1] + residual)
    cusum_negative.append(s_neg)

# Remover o primeiro elemento (inicialização)
cusum_positive = cusum_positive[1:]
cusum_negative = cusum_negative[1:]

# Adicionar ao DataFrame para facilitar o manuseio
residuals_df = pd.DataFrame({
    'Residuals': residuals,
    'CUSUM_Positive': cusum_positive,
    'CUSUM_Negative': cusum_negative
}, index=residuals.index)

# Exibir as primeiras linhas para verificação
display(residuals_df.head())


In [ ]:
# Plotar CUSUM Positivo e Negativo
plt.figure(figsize=(14, 7))
plt.plot(residuals_df['CUSUM_Positive'], label='CUSUM Positivo', color='green')
plt.plot(residuals_df['CUSUM_Negative'], label='CUSUM Negativo', color='red')
plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.title('CUSUM dos Resíduos')
plt.xlabel('Observações')
plt.ylabel('Cumulative Sum')
plt.legend()
plt.show()


# Indicador de Volatilidade

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# --- Parte anterior: Cálculo dos Resíduos ---
# Converter y para pandas Series (certifique-se de que 'y' está definido)
y_series = pd.Series(y, name='y_true')

# Verificar se Y_pred é um DataFrame com uma única coluna
if isinstance(Y_pred, pd.DataFrame):
    if Y_pred.shape[1] == 1:
        # Converter para Series sem resetar o índice
        Y_pred_series = Y_pred.iloc[:, 0]
        Y_pred_series.name = 'y_pred'
        print(f"Convertido Y_pred para Series com comprimento {len(Y_pred_series)}.")
    else:
        raise ValueError("Y_pred possui mais de uma coluna. Por favor, selecione a coluna correta para as previsões.")
elif isinstance(Y_pred, pd.Series):
    Y_pred_series = Y_pred
    Y_pred_series.name = 'y_pred'
    print(f"Y_pred já é uma Series com comprimento {len(Y_pred_series)}.")
else:
    raise TypeError("Y_pred deve ser uma pandas Series ou DataFrame.")

# Verificar comprimentos
len_y = len(y_series)
len_Y_pred = len(Y_pred_series)

print(f"Comprimento de y_series: {len_y}")
print(f"Comprimento de Y_pred_series: {len_Y_pred}")

# Calcular a diferença de comprimento
diff = len_Y_pred - len_y

if diff > 0:
    print(f"Y_pred_series é {diff} pontos maior que y_series. Truncando os últimos {diff} pontos de Y_pred_series.")
    Y_pred_aligned = Y_pred_series.iloc[:-diff]
    y_aligned = y_series
elif diff < 0:
    print(f"Y_pred_series é {abs(diff)} pontos menor que y_series. Truncando os últimos {abs(diff)} pontos de y_series.")
    y_aligned = y_series.iloc[:len_Y_pred]
    Y_pred_aligned = Y_pred_series
else:
    print("y_series e Y_pred_series já estão alinhadas em comprimento.")
    y_aligned = y_series
    Y_pred_aligned = Y_pred_series

print(f"Comprimento de y_aligned: {len(y_aligned)}")
print(f"Comprimento de Y_pred_aligned: {len(Y_pred_aligned)}")

# Calcular os resíduos
residuals = y_aligned - Y_pred_aligned

# Exibir as primeiras linhas dos resíduos
display(residuals.head())

# --- Implementação do Indicador de Volatilidade ---
# Definir os parâmetros para os indicadores
window_size_std = 20  # Período para Desvio Padrão Móvel
window_size_bb = 20   # Período para Bollinger Bands
num_std_dev_bb = 2    # Número de desvios padrão para Bollinger Bands
span_ewma = 20        # Span para EWMA
window_size_atr = 14  # Período para ATR

# --- 1. Desvio Padrão Móvel ---
series['Rolling_STD'] = series['price_close'].rolling(window=window_size_std).std()

# Verificar se a coluna foi criada
if 'Rolling_STD' in series.columns:
    print("Desvio Padrão Móvel calculado com sucesso.")
else:
    print("Erro ao calcular o Desvio Padrão Móvel.")

# --- 2. Bollinger Bands ---
rolling_mean = series['price_close'].rolling(window=window_size_bb).mean()
rolling_std_bb = series['price_close'].rolling(window=window_size_bb).std()

bollinger_upper = rolling_mean + (rolling_std_bb * num_std_dev_bb)
bollinger_lower = rolling_mean - (rolling_std_bb * num_std_dev_bb)

series['Rolling_Mean'] = rolling_mean
series['Bollinger_Upper'] = bollinger_upper
series['Bollinger_Lower'] = bollinger_lower

# Verificar se as colunas foram criadas
required_bb_columns = ['Rolling_Mean', 'Bollinger_Upper', 'Bollinger_Lower']
if all(col in series.columns for col in required_bb_columns):
    print("Bollinger Bands calculadas com sucesso.")
else:
    missing_bb = [col for col in required_bb_columns if col not in series.columns]
    print(f"Erro ao calcular Bollinger Bands. Colunas faltantes: {missing_bb}")

# --- 3. EWMA Desvio Padrão ---
series['EWMA_STD'] = series['price_close'].ewm(span=span_ewma, adjust=False).std()

if 'EWMA_STD' in series.columns:
    print("EWMA Desvio Padrão calculado com sucesso.")
else:
    print("Erro ao calcular o EWMA Desvio Padrão.")

# --- 4. Average True Range (ATR) ---
required_atr_columns = ['price_high', 'price_low', 'price_close']
if all(col in series.columns for col in required_atr_columns):
    high_low = series['price_high'] - series['price_low']
    high_prev_close = (series['price_high'] - series['price_close'].shift()).abs()
    low_prev_close = (series['price_low'] - series['price_close'].shift()).abs()
    true_range = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1)
    series['ATR'] = true_range.rolling(window=window_size_atr).mean()
    print("Average True Range (ATR) calculado com sucesso.")
else:
    missing_atr = [col for col in required_atr_columns if col not in series.columns]
    print(f"Não foi possível calcular o ATR. Colunas faltantes: {missing_atr}")

# --- 5. Visualizar os Indicadores de Volatilidade ---

# a. Desvio Padrão Móvel
plt.figure(figsize=(14, 7))
plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
plt.plot(series['Rolling_STD'], label=f'Desvio Padrão Móvel ({window_size_std} períodos)', color='orange')
plt.title('Desvio Padrão Móvel da Série de Preços')
plt.xlabel('Data')
plt.ylabel('Preço / Volatilidade')
plt.legend()
plt.show()

# b. Bollinger Bands
plt.figure(figsize=(14, 7))
plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
plt.plot(series['Rolling_Mean'], label=f'Média Móvel ({window_size_bb} períodos)', color='orange')
plt.plot(series['Bollinger_Upper'], label='Bollinger Upper', color='green')
plt.plot(series['Bollinger_Lower'], label='Bollinger Lower', color='red')
plt.fill_between(series.index, series['Bollinger_Lower'], series['Bollinger_Upper'], color='lightgray')
plt.title('Bollinger Bands da Série de Preços')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.legend()
plt.show()

# c. EWMA Desvio Padrão
plt.figure(figsize=(14, 7))
plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
plt.plot(series['EWMA_STD'], label=f'EWMA Desvio Padrão (span={span_ewma})', color='purple')
plt.title('EWMA Desvio Padrão da Série de Preços')
plt.xlabel('Data')
plt.ylabel('Preço / Volatilidade')
plt.legend()
plt.show()

# d. ATR (Opcional)
if 'ATR' in series.columns:
    plt.figure(figsize=(14, 7))
    plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
    plt.plot(series['ATR'], label=f'ATR ({window_size_atr} períodos)', color='magenta')
    plt.title('Average True Range (ATR) da Série de Preços')
    plt.xlabel('Data')
    plt.ylabel('Preço / ATR')
    plt.legend()
    plt.show()

# --- 6. Integrar os Indicadores no Modelo de Regressão ---

# a. Selecionar as features disponíveis
features = ['Rolling_STD', 'Rolling_Mean', 'Bollinger_Upper', 'Bollinger_Lower', 'EWMA_STD']

# Adicionar 'ATR' se estiver disponível
if 'ATR' in series.columns:
    features.append('ATR')

# Verificar quais features estão presentes no DataFrame
available_features = [feature for feature in features if feature in series.columns]
missing_features = [feature for feature in features if feature not in series.columns]

if missing_features:
    print(f"As seguintes features estão faltando e serão ignoradas: {missing_features}")

print(f"Features disponíveis para modelagem: {available_features}")

# b. Preparar os Dados para o Modelo
X_features = series[available_features].dropna()
print(f"\nNúmero de linhas em X_features após dropna(): {len(X_features)}")

# Encontrar os índices comuns entre X_features e y_aligned
common_indices = X_features.index.intersection(y_aligned.index)
print(f"Número de índices comuns: {len(common_indices)}")

# Selecionar apenas os índices comuns
X_features_aligned = X_features.loc[common_indices]
y_features_aligned = y_aligned.loc[common_indices]

print(f"Comprimento de X_features_aligned: {len(X_features_aligned)}")
print(f"Comprimento de y_features_aligned: {len(y_features_aligned)}")

# Verificar se os índices estão alinhados
if len(X_features_aligned) != len(y_features_aligned):
    print("Os dados das features e da variável alvo ainda não estão alinhados.")
else:
    print("X_features e y_features estão alinhados corretamente.")

# c. Preprocessamento e PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features_aligned)

pca = PCA()
X_pca = pca.fit_transform(X_scaled)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
print("\nVariância explicada acumulada:", cumulative_variance)

# Determinar o número de componentes para 95% da variância
n_components_var = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Número ótimo de componentes (95% da variância explicada): {n_components_var}")

# Reduzir a dimensionalidade
pca_reduced = PCA(n_components=n_components_var)
X_reduced = pca_reduced.fit_transform(X_scaled)
print(f"Forma da matriz após PCA: {X_reduced.shape}")

# d. Modelagem com Regressão Linear
model = LinearRegression()
scores = cross_val_score(model, X_reduced, y_features_aligned, cv=5, scoring='neg_mean_squared_error')
mse_mean = -np.mean(scores)
print(f"MSE médio (5-fold CV): {mse_mean:.4f}")


# Política das 3 Barreiras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# --- Parte anterior: Cálculo dos Resíduos ---
# Converter y para pandas Series (certifique-se de que 'y' está definido)
y_series = pd.Series(y, name='y_true')

# Verificar se Y_pred é um DataFrame com uma única coluna
if isinstance(Y_pred, pd.DataFrame):
    if Y_pred.shape[1] == 1:
        # Converter para Series sem resetar o índice
        Y_pred_series = Y_pred.iloc[:, 0]
        Y_pred_series.name = 'y_pred'
        print(f"Convertido Y_pred para Series com comprimento {len(Y_pred_series)}.")
    else:
        raise ValueError("Y_pred possui mais de uma coluna. Por favor, selecione a coluna correta para as previsões.")
elif isinstance(Y_pred, pd.Series):
    Y_pred_series = Y_pred
    Y_pred_series.name = 'y_pred'
    print(f"Y_pred já é uma Series com comprimento {len(Y_pred_series)}.")
else:
    raise TypeError("Y_pred deve ser uma pandas Series ou DataFrame.")

# Verificar comprimentos
len_y = len(y_series)
len_Y_pred = len(Y_pred_series)

print(f"Comprimento de y_series: {len_y}")
print(f"Comprimento de Y_pred_series: {len_Y_pred}")

# Calcular a diferença de comprimento
diff = len_Y_pred - len_y

if diff > 0:
    print(f"Y_pred_series é {diff} pontos maior que y_series. Truncando os últimos {diff} pontos de Y_pred_series.")
    Y_pred_aligned = Y_pred_series.iloc[:-diff]
    y_aligned = y_series
elif diff < 0:
    print(f"Y_pred_series é {abs(diff)} pontos menor que y_series. Truncando os últimos {abs(diff)} pontos de y_series.")
    y_aligned = y_series.iloc[:len_Y_pred]
    Y_pred_aligned = Y_pred_series
else:
    print("y_series e Y_pred_series já estão alinhadas em comprimento.")
    y_aligned = y_series
    Y_pred_aligned = Y_pred_series

print(f"Comprimento de y_aligned: {len(y_aligned)}")
print(f"Comprimento de Y_pred_aligned: {len(Y_pred_aligned)}")

# Calcular os resíduos
residuals = y_aligned - Y_pred_aligned

# Adicionar os resíduos ao DataFrame 'series'
series['Residuals'] = residuals

# Verificar se todos os índices de residuals estão em series
missing_indices = residuals.index.difference(series.index)
if not missing_indices.empty:
    print(f"\nÍndices faltantes em 'series' que estão em 'residuals': {missing_indices.tolist()}")
    # Remover esses índices de residuals
    residuals = residuals.drop(index=missing_indices)
    # Atualizar o DataFrame 'series' para refletir a remoção
    series = series.drop(index=missing_indices)
    print(f"Residuals após remoção dos índices faltantes: {len(residuals)}")
else:
    print("\nTodos os índices de 'residuals' estão presentes em 'series'.")

# --- Implementação do Indicador de Volatilidade ---
# Definir os parâmetros para os indicadores
window_size_std = 20  # Período para Desvio Padrão Móvel
window_size_bb = 20   # Período para Bollinger Bands
num_std_dev_bb = 2    # Número de desvios padrão para Bollinger Bands
span_ewma = 20        # Span para EWMA
window_size_atr = 14  # Período para ATR

# --- 1. Desvio Padrão Móvel ---
series['Rolling_STD'] = series['price_close'].rolling(window=window_size_std).std()

# Verificar se a coluna foi criada
if 'Rolling_STD' in series.columns:
    print("Desvio Padrão Móvel calculado com sucesso.")
else:
    print("Erro ao calcular o Desvio Padrão Móvel.")

# --- 2. Bollinger Bands ---
rolling_mean = series['price_close'].rolling(window=window_size_bb).mean()
rolling_std_bb = series['price_close'].rolling(window=window_size_bb).std()

bollinger_upper = rolling_mean + (rolling_std_bb * num_std_dev_bb)
bollinger_lower = rolling_mean - (rolling_std_bb * num_std_dev_bb)

series['Rolling_Mean'] = rolling_mean
series['Bollinger_Upper'] = bollinger_upper
series['Bollinger_Lower'] = bollinger_lower

# Verificar se as colunas foram criadas
required_bb_columns = ['Rolling_Mean', 'Bollinger_Upper', 'Bollinger_Lower']
if all(col in series.columns for col in required_bb_columns):
    print("Bollinger Bands calculadas com sucesso.")
else:
    missing_bb = [col for col in required_bb_columns if col not in series.columns]
    print(f"Erro ao calcular Bollinger Bands. Colunas faltantes: {missing_bb}")

# --- 3. EWMA Desvio Padrão ---
series['EWMA_STD'] = series['price_close'].ewm(span=span_ewma, adjust=False).std()

if 'EWMA_STD' in series.columns:
    print("EWMA Desvio Padrão calculado com sucesso.")
else:
    print("Erro ao calcular o EWMA Desvio Padrão.")

# --- 4. Average True Range (ATR) ---
required_atr_columns = ['price_high', 'price_low', 'price_close']
if all(col in series.columns for col in required_atr_columns):
    high_low = series['price_high'] - series['price_low']
    high_prev_close = (series['price_high'] - series['price_close'].shift()).abs()
    low_prev_close = (series['price_low'] - series['price_close'].shift()).abs()
    true_range = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1)
    series['ATR'] = true_range.rolling(window=window_size_atr).mean()
    print("Average True Range (ATR) calculado com sucesso.")
else:
    missing_atr = [col for col in required_atr_columns if col not in series.columns]
    print(f"Não foi possível calcular o ATR. Colunas faltantes: {missing_atr}")

# --- 5. Visualizar os Indicadores de Volatilidade ---
# a. Desvio Padrão Móvel
plt.figure(figsize=(14, 7))
plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
plt.plot(series['Rolling_STD'], label=f'Desvio Padrão Móvel ({window_size_std} períodos)', color='orange')
plt.title('Desvio Padrão Móvel da Série de Preços')
plt.xlabel('Data')
plt.ylabel('Preço / Volatilidade')
plt.legend()
plt.show()

# b. Bollinger Bands
plt.figure(figsize=(14, 7))
plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
plt.plot(series['Rolling_Mean'], label=f'Média Móvel ({window_size_bb} períodos)', color='orange')
plt.plot(series['Bollinger_Upper'], label='Bollinger Upper', color='green')
plt.plot(series['Bollinger_Lower'], label='Bollinger Lower', color='red')
plt.fill_between(series.index, series['Bollinger_Lower'], series['Bollinger_Upper'], color='lightgray')
plt.title('Bollinger Bands da Série de Preços')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.legend()
plt.show()

# c. EWMA Desvio Padrão
plt.figure(figsize=(14, 7))
plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
plt.plot(series['EWMA_STD'], label=f'EWMA Desvio Padrão (span={span_ewma})', color='purple')
plt.title('EWMA Desvio Padrão da Série de Preços')
plt.xlabel('Data')
plt.ylabel('Preço / Volatilidade')
plt.legend()
plt.show()

# d. ATR (Opcional)
if 'ATR' in series.columns:
    plt.figure(figsize=(14, 7))
    plt.plot(series['price_close'], label='Preço de Fechamento', color='blue')
    plt.plot(series['ATR'], label=f'ATR ({window_size_atr} períodos)', color='magenta')
    plt.title('Average True Range (ATR) da Série de Preços')
    plt.xlabel('Data')
    plt.ylabel('Preço / ATR')
    plt.legend()
    plt.show()

# --- 6. Calcular o CUSUM dos Resíduos ---
# Parâmetros para o CUSUM
k = 0.5  # Tolerância, ajuste conforme necessário

# Inicializar as colunas de CUSUM positivo e negativo como float
series['CUSUM_Positive'] = 0.0
series['CUSUM_Negative'] = 0.0

# Calcular o CUSUM usando iterrows()
for i in range(1, len(series)):
    current_index = series.index[i]
    previous_index = series.index[i-1]
    
    try:
        # Calcular CUSUM Positivo
        series.at[current_index, 'CUSUM_Positive'] = max(
            0.0, 
            series.at[previous_index, 'CUSUM_Positive'] + residuals.loc[current_index] - k
        )
        
        # Calcular CUSUM Negativo
        series.at[current_index, 'CUSUM_Negative'] = min(
            0.0, 
            series.at[previous_index, 'CUSUM_Negative'] + residuals.loc[current_index] + k
        )
    except KeyError as e:
        print(f"Erro ao acessar o índice {current_index}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao calcular CUSUM para o índice {current_index}: {e}")

# --- 7. Definir as Barreiras ---
# Definir os múltiplos para as barreiras
barrier_levels = {
    'alert': 1,      # 1x a volatilidade
    'action': 2,     # 2x a volatilidade
    'critical': 3    # 3x a volatilidade
}

# Calcular as barreiras positivas e negativas
for level, multiplier in barrier_levels.items():
    series[f'Barreira_Pos_{level}'] = series['Rolling_STD'] * multiplier
    series[f'Barreira_Neg_{level}'] = -series['Rolling_STD'] * multiplier

# --- 8. Aplicar a Política das 3 Barreiras ---
# Inicializar uma coluna para o gatilho
series['Gatilho'] = None

# Função para aplicar a política das 3 barreiras
def aplicar_politica(row):
    # CUSUM Positivo
    if row['CUSUM_Positive'] > row['Barreira_Pos_alert']:
        if row['CUSUM_Positive'] > row['Barreira_Pos_critical']:
            return 'Crítica Positiva'
        elif row['CUSUM_Positive'] > row['Barreira_Pos_action']:
            return 'Ação Positiva'
        else:
            return 'Alerta Positivo'
    # CUSUM Negativo
    elif row['CUSUM_Negative'] < row['Barreira_Neg_alert']:
        if row['CUSUM_Negative'] < row['Barreira_Neg_critical']:
            return 'Crítica Negativa'
        elif row['CUSUM_Negative'] < row['Barreira_Neg_action']:
            return 'Ação Negativa'
        else:
            return 'Alerta Negativo'
    else:
        return None

# Aplicar a política das 3 barreiras
series['Gatilho'] = series.apply(aplicar_politica, axis=1)

# --- 9. Visualizar os Gatilhos ---
# a. CUSUM Positivo com Barreiras e Gatilhos
plt.figure(figsize=(14, 7))
plt.plot(series['CUSUM_Positive'], label='CUSUM Positivo', color='green')
plt.plot(series['Barreira_Pos_alert'], label='Barreira Positiva - Alerta', linestyle='--', color='orange')
plt.plot(series['Barreira_Pos_action'], label='Barreira Positiva - Ação', linestyle='--', color='red')
plt.plot(series['Barreira_Pos_critical'], label='Barreira Positiva - Crítica', linestyle='--', color='purple')

# Marcadores para os gatilhos positivos
gatilhos_pos = series[series['Gatilho'].isin(['Alerta Positivo', 'Ação Positiva', 'Crítica Positiva'])]
plt.scatter(
    gatilhos_pos.index, 
    gatilhos_pos['CUSUM_Positive'], 
    c=gatilhos_pos['Gatilho'].map({
        'Alerta Positivo': 'yellow', 
        'Ação Positiva': 'orange', 
        'Crítica Positiva': 'red'
    }), 
    label='Gatilho Positivo', 
    marker='o'
)

plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.title('CUSUM Positivo com Barreiras e Gatilhos')
plt.xlabel('Data')
plt.ylabel('Cumulative Sum')
plt.legend()
plt.show()

# b. CUSUM Negativo com Barreiras e Gatilhos
plt.figure(figsize=(14, 7))
plt.plot(series['CUSUM_Negative'], label='CUSUM Negativo', color='red')
plt.plot(series['Barreira_Neg_alert'], label='Barreira Negativa - Alerta', linestyle='--', color='orange')
plt.plot(series['Barreira_Neg_action'], label='Barreira Negativa - Ação', linestyle='--', color='blue')
plt.plot(series['Barreira_Neg_critical'], label='Barreira Negativa - Crítica', linestyle='--', color='purple')

# Marcadores para os gatilhos negativos
gatilhos_neg = series[series['Gatilho'].isin(['Alerta Negativo', 'Ação Negativa', 'Crítica Negativa'])]
plt.scatter(
    gatilhos_neg.index, 
    gatilhos_neg['CUSUM_Negative'], 
    c=gatilhos_neg['Gatilho'].map({
        'Alerta Negativo': 'yellow', 
        'Ação Negativa': 'blue', 
        'Crítica Negativa': 'purple'
    }), 
    label='Gatilho Negativo', 
    marker='o'
)

plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.title('CUSUM Negativo com Barreiras e Gatilhos')
plt.xlabel('Data')
plt.ylabel('Cumulative Sum')
plt.legend()
plt.show()

# --- 10. Exibir os Eventos de Gatilho ---
# Filtrar os eventos de gatilho
eventos_gatilho = series[series['Gatilho'].notnull()][['Gatilho']]

print("Eventos de Gatilho Detectados:")
display(eventos_gatilho)
